### 1. Mount Google Drive
Mounts Google Drive to access the processed Other CS dataset.

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### 2. Initialize Qdrant Client
Initializes the connection to the Qdrant Cloud vector database. This client will be used to create the `other_cs` collection and upload vectors.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

qdrant_client = QdrantClient(
     url="Enter your api URL***",
     api_key="Enter your Api key***"
)


### 3. Inspect Processed Data
Reads and prints the first line of the processed Other CS dataset (`other_cs.jsonl`) to verify the data format before indexing.

In [5]:
import json

path = "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/other_cs.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(json.loads(line))
        break


{'openalex_id': 'https://openalex.org/W1803273808', 'doi': 'https://doi.org/10.4135/9781036235611', 'title': 'The Coding Manual for Qualitative Researchers', 'publication_year': 2025, 'publication_date': '2025-01-01', 'authors': [{'author_id': 'https://openalex.org/A5058231379', 'name': 'Johnny Saldaña'}], 'concepts': [{'id': 'https://openalex.org/C179518139', 'name': 'Coding (social sciences)'}, {'id': 'https://openalex.org/C2780031656', 'name': 'Glossary'}, {'id': 'https://openalex.org/C73231260', 'name': 'Tunstall coding'}, {'id': 'https://openalex.org/C130811719', 'name': 'Shannon–Fano coding'}, {'id': 'https://openalex.org/C60603091', 'name': 'Variable-length code'}, {'id': 'https://openalex.org/C41008148', 'name': 'Computer science'}, {'id': 'https://openalex.org/C135534801', 'name': 'Context-adaptive variable-length coding'}, {'id': 'https://openalex.org/C175732694', 'name': 'Context-adaptive binary arithmetic coding'}, {'id': 'https://openalex.org/C33923547', 'name': 'Mathemati

### 4. Full Indexing Pipeline (Other CS Domain)
This is the main ingestion script for the Other CS domain:
1.  **Config**: Sets the collection name (`other_cs`), batch size, and vector size.
2.  **Setup**: Creates the Qdrant collection if it doesn't exist.
3.  **Ingest**:
    *   Reads the `other_cs.jsonl` file.
    *   Generates embeddings using `all-MiniLM-L6-v2`.
    *   Constructs the payload with metadata.
    *   Uploads points in batches to Qdrant.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
import json
from tqdm import tqdm

# ---------------- CONFIG ----------------
COLLECTION = "other_cs"   # ✅ NEW COLLECTION
BATCH_SIZE = 64             # safer for cloud
VECTOR_SIZE = 384

# ---------------- MODEL -----------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ---------------- QDRANT ----------------
client = QdrantClient(
    url="Enter your api URL***",
    api_key="Enter your Api key***"
    timeout=60  # ✅ IMPORTANT
)

# ---------------- COLLECTION SETUP ----------------
if not client.collection_exists(COLLECTION):
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config=VectorParams(
            size=VECTOR_SIZE,
            distance=Distance.COSINE
        )
    )

# ---------------- INGEST ----------------
points = []
idx = 0

with open(
    "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/other_cs.jsonl",
    "r",
    encoding="utf-8"
) as f:

    for line in tqdm(f, desc="Indexing Other CS Papers"):
        paper = json.loads(line)

        text = f"{paper.get('title','')} {paper.get('abstract','')}".strip()
        if len(text) < 30:
            continue

        vector = model.encode(text).tolist()

        payload = {
            "openalex_id": paper.get("openalex_id"),
            "doi": paper.get("doi"),
            "title": paper.get("title"),
            "abstract": paper.get("abstract"),
            "publication_year": paper.get("publication_year"),
            "publication_date": paper.get("publication_date"),
            "venue": paper.get("venue"),
            "citation_count": paper.get("citation_count"),
            "is_open_access": paper.get("is_open_access"),
            "oa_status": paper.get("oa_status"),
            "url": paper.get("url"),

            # 🔥 STRUCTURED FIELDS
            "authors": [
                {
                    "id": a.get("author_id"),
                    "name": a.get("name")
                } for a in paper.get("authors", [])
            ],

            "concepts": [
                {
                    "id": c.get("id"),
                    "name": c.get("name")
                } for c in paper.get("concepts", [])
            ]
        }

        points.append(
            PointStruct(
                id=idx,
                vector=vector,
                payload=payload
            )
        )
        idx += 1

        if len(points) == BATCH_SIZE:
            client.upsert(
                collection_name=COLLECTION,
                points=points,
                wait=False  # ✅ avoids timeout
            )
            points = []

# Final flush
if points:
    client.upsert(
        collection_name=COLLECTION,
        points=points,
        wait=False
    )

print("✅ FULL Other CS COLLECTION INDEXED SUCCESSFULLY")

### 5. Test Retrieval (Other CS Domain)
Performs a sanity check on the indexed `other_cs` collection:
1.  Encodes a test query ("What are the open research gaps in deep Learning?").
2.  Searches the collection for the top 5 nearest neighbors.
3.  Prints the detailed results to verify retrieval quality.

In [ ]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

client = QdrantClient(
   url="Enter your api URL***",
    api_key="Enter your Api key***"
)

query = "What are the open research gaps in deep Learning?"
vector = model.encode(query).tolist()

results = client.query_points(
    collection_name="other_cs"   ,
    query=vector,
    limit=5,
    with_payload=True
)

print(f"Returned points: {len(results.points)}")

for i, hit in enumerate(results.points, start=1):
    p = hit.payload

    print(f"\n--- Result {i} ---")
    print("Score:", round(hit.score, 4))
    print("Title:", p.get("title"))
    print("DOI:", p.get("doi"))
    print("Year:", p.get("publication_year"))
    print("Venue:", p.get("venue"))
    print("Citations:", p.get("citation_count"))
    print("Open Access:", p.get("is_open_access"), "| Status:", p.get("oa_status"))
    print("URL:", p.get("url"))

    # ✅ SAFE abstract handling
    abstract = p.get("abstract")
    if abstract:
        print("Abstract:", abstract[:600])
    else:
        print("Abstract: ❌ Not available")

    # ✅ Authors
    authors = p.get("authors", [])
    if authors:
        print("Authors:", ", ".join(a["name"] for a in authors[:5]))
    else:
        print("Authors: ❌ Not available")

    # ✅ Concepts
    concepts = p.get("concepts", [])
    if concepts:
        print("Concepts:", ", ".join(c["name"] for c in concepts[:6]))
    else:
        print("Concepts: ❌ Not available")

Returned points: 5

--- Result 1 ---
Score: 0.5708
Title: OpenAI’s ‘deep research’ tool: is it useful for scientists?
DOI: https://doi.org/10.1038/d41586-025-00377-9
Year: 2025
Venue: Nature
Citations: 13
Open Access: False | Status: closed
URL: https://openalex.org/W4407199411
Abstract: ❌ Not available
Authors: Nicola Jones
Concepts: Data science, Computer science, Engineering ethics, Management science, Engineering

--- Result 2 ---
Score: 0.5376
Title: Google’s AI co-scientist and OpenAI’s deep research: new partners in health research?
DOI: https://doi.org/10.1093/eurjcn/zvaf066
Year: 2025
Venue: European Journal of Cardiovascular Nursing
Citations: 2
Open Access: True | Status: hybrid
URL: https://openalex.org/W4412491094
Abstract: ❌ Not available
Authors: Philip Moons, Bei Dou, Chloé P Desmedt
Concepts: Medicine, Data science, Cognitive science, Psychology, Computer science

--- Result 3 ---
Score: 0.5192
Title: Scientists flock to DeepSeek: how they’re using the blockbuster AI m